<a href="https://colab.research.google.com/github/amyjchao/data-course-sample/blob/main/ACHW3_Collaborative_filtering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3-實作 User-based Filtering 推薦算法

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-09 00:19:12--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  8.92MB/s    in 1.7s    

2022-01-09 00:19:14 (8.92 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-09 00:19:15--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.65MB/s    in 1.5s    

202

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
import string

In [7]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [8]:
#remove same title items
metadata = metadata[['asin', 'brand', 'title','rank','description']]
metadat = metadata.drop_duplicates('title')

In [9]:
#split category and rank in the original rank column
metadata['category'] = metadata['rank'].str.split(' in ').str[1]
metadata['rank'] = metadata['rank'].str.split(expand = True)[0]
metadata['category'] = metadata['category'].str[:-2]
metadata['category'].unique()
metadata = metadata[metadata['category'] == 'Beauty & Personal Care']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
#turn string to int
metadata['rank'] = metadata['rank'].str.replace(',','')
metadata['rank'] = pd.to_numeric(metadata['rank'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#only need these columns
metadata = metadata[['asin','title','rank','category']]

## 資料切分

In [12]:

ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [19]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
import datetime

### 測試邏輯

In [ ]:
'''
上次發現評分>=4的話，recall會提高許多，但是上次是只有留著rating>=4的
這次是用mean來看評分>=4 -> recall 0.05
              評分>=3 -> recall 0.1372
->>不同的方法要用不同的tuning parameter
工作坊是有位同學說篩掉size=1的，-> recall沒有變
product_rank 4049 rows
combine 1659 rows
size_df 1659 rows -> 551 rows (篩掉size=1)
mean_df 1117 rows -> 338 rows (篩掉size=1)
'''
product_rank = metadata[['asin','rank']]
train_ratings = ratings_trainings[(ratings_trainings['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']
combine = train_ratings.set_index('asin').join(product_rank.set_index('asin'))
overall_df = combine.groupby('asin').aggregate({'overall':['mean', 'size']})['overall'].reset_index()
size_df = overall_df.sort_values(['size'], ascending=False)
mean_df = size_df[(size_df['mean']>=3)]
#size_df = size_df[(size_df['size']<2) == False]
#mean_df = size_df[(size_df['mean']>=3)]
#print(combine.head)
#print(overall_df.head)
print(overall_df.head)

<bound method NDFrame.head of            asin  mean  size
0    B00N6FCRYS   1.0     1
1    B00Z5VI9MK   2.0     1
2    B00ZSOV3Q8   5.0     1
3    B010GG8JGO   5.0     1
4    B010SV3L3I   5.0     2
..          ...   ...   ...
146  B01GUDFZ2I   5.0     1
147  B01H3IVSAC   5.0     1
148  B01H4BQUZG   5.0     1
149  B01HEYYRIK   3.0     1
150  B01HI1YKIG   4.0     1

[151 rows x 3 columns]>


### User-based

In [ ]:
# header: user_id,item_id,rating,timestamp

def user_recommender(training_data, users=[], k=10,wk=52,s=3):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']
    
    # preparing top item list
    product_rank = metadata[['asin','rank']]

    train_ratings = ratings_trainings[(ratings_trainings['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']
    combine = train_ratings.set_index('asin').join(product_rank.set_index('asin'))
    overall_df = combine.groupby('asin').aggregate({'overall':['mean', 'size']})['overall'].reset_index()
    size_df = overall_df.sort_values(['size'], ascending=False)
    mean_df = size_df[(size_df['mean']>=s)]

    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
      row = dict(row)
      user = row['reviewerID']
      item = row['asin']
      rating = float(row['overall'])

      user_to_items[user][item] = rating

    #print("total users before filtering: ", len(user_to_items))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = True
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
      ratings = user_to_items[user]
      if remove_obscure_user and len(ratings) < user_rating_threshold:
          del user_to_items[user]

    #print("total users  after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
      for item, rating in items.items():
          item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
      if len(user_ratings) > 1:
          # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
          for user1, user2 in combinations(user_ratings.keys(), 2):
              xy = user_ratings[user1] * user_ratings[user2]
              xx = user_ratings[user1] ** 2
              yy = user_ratings[user2] ** 2
              pre_user_similarity[user1][user2][0] += xy
              pre_user_similarity[user1][user2][1] += xx
              pre_user_similarity[user1][user2][2] += yy

              pre_user_similarity[user2][user1][0] += xy
              pre_user_similarity[user2][user1][1] += xx
              pre_user_similarity[user2][user1][2] += yy
      index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
      user_similarity_order = []
      for dst_user, val in pre_user_similarity[src_user].items():
          xy = val[0]
          xx = val[1]
          yy = val[2]
          div = ((xx*yy) ** 0.5)
          if div == 0:
              continue
          similarity = xy / div
          if similarity < 0:
              continue
          for i, s in enumerate(user_similarity_order):
              target_similarity = s[1]
              if target_similarity < similarity:
                  user_similarity_order.insert(i, (dst_user, similarity))
                  break
          else:
              user_similarity_order.append((dst_user, similarity))
      user_similarity[src_user] = user_similarity_order

    recommendation = {}
    for user in users:
      if user in user_similarity:
          sim_users = user_similarity[user]
          recommended_items = []
          recommended_items_set = set()
          user_have_rated = set(user_to_items[user])
          stop_recommend = False
          for sim_user, _ in sim_users:
              items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
              for item, _ in items_from_sim_user:
                  if item not in user_have_rated and item not in recommended_items_set:
                      recommended_items.append(item)
                      recommended_items_set.add(item)
                  if len(recommended_items) >= k:
                      stop_recommend = True
                      break
              if stop_recommend:
                  break
          recommendation[user] = recommended_items
      else:
        topitem = mean_df['asin'].tolist()[:k]
        recommendation[user] = topitem

    return recommendation


### Item-based

In [ ]:
def item_recommender(training_data, users=[], k=10, wk=52, s=3):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']
    
    # preparing top item list
    product_rank = metadata[['asin','rank']]
    train_ratings = ratings_trainings[(ratings_trainings['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']
    combine = train_ratings.set_index('asin').join(product_rank.set_index('asin'))
    overall_df = combine.groupby('asin').aggregate({'overall':['mean', 'size']})['overall'].reset_index()
    size_df = overall_df.sort_values(['size'], ascending=False)
    mean_df = size_df[(size_df['mean']>=s)]

    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    #print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    #print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    #print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    #print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        if len(items) < 1:
          items = mean_df['asin'].tolist()[:k]
          
        recommendation[user] = items
    return recommendation    


### 套件surprise

In [13]:
pip install surprise

     |████████████████████████████████| 11.8 MB 14.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619433 sha256=82cee4988adc6a50b736837c46a744231607596f519c60936f2ae03cce34a4ac
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [14]:
from collections import defaultdict

def get_top_n(predictions, n=10):

    # 首先將預測資料對應到 user 
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # 將預測值進行排序，並且選取最高的 n 個值回傳
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def recommend_items(model, user, items_user_not_rated, ratings_mean, k):
    
    # 取出測試資料
    user_recommend_data = [(user, item, ratings_mean) for item in items_user_not_rated]

    # 預測
    predictions = model.test(user_recommend_data)
    top_n = get_top_n(predictions, n=k)
    return [top[0] for top in top_n[user]]
    

In [15]:
from scipy.sparse import csr_matrix
from surprise import Reader
from surprise import Dataset
from surprise import SVD, SVDpp
from surprise import accuracy
import numpy as np

def surprise_recommender(training_data, users=[], k=10, wk=52, s=3):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    mf-based, surprise
    '''
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']

    # 取出評分矩陣所需的訓練資料
    training_data = training_data[["asin","reviewerID","overall"]]
    item_set_trainings = set(training_data["asin"].to_list())

    # 轉換訓練資料為套件需要的格式
    reader = Reader(rating_scale = (min(training_data["overall"]), max(training_data["overall"])))
    train_data = Dataset.load_from_df(training_data, reader).build_full_trainset()

    # 訓練模型
    model = SVD()
    # model = SVDpp()
    model.fit(train_data)

    # 推薦
    recommendations = {user: recommend_items(model, user, item_set_trainings - set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list()), s, k) for user in users}
    
    return recommendations

# 推薦成果：可看出冷啟動問題嚴重，針對未曾出現的使用者無法有鑑別的個別推薦，實務上可能會主動請使用者先評分再訓練進行推薦

## 結果評估

In [16]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

## 各方式評估

In [17]:
import numpy as np

### User-based

In [ ]:
#score and wks
rates = range(1,6)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for rate in rates:
    for wk in wks:    
        ratings_by_user = user_recommender(ratings_trainings, users, k=10, wk=wk, s=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("s = %d, weeks = %d, score = %.5f" %(rate, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(rates, name='rate'))

rate,1,2,3,4,5
Weeks,,,,,
1,0.155932,0.155932,0.155932,0.159322,0.035593
2,0.140678,0.140678,0.140678,0.072881,0.016949
4,0.157627,0.157627,0.157627,0.035593,0.022034
8,0.152542,0.152542,0.152542,0.030508,0.028814
12,0.133898,0.133898,0.133898,0.020339,0.011864
24,0.096610,0.096610,0.096610,0.016949,0.011864
52,0.098305,0.098305,0.098305,0.098305,0.000000
104,0.083051,0.083051,0.083051,0.096610,0.000000


In [ ]:
#wks and ks
ks = range(10,51,10)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for k in ks:
    for wk in wks:    
        ratings_by_user = user_recommender(ratings_trainings, users, k=k, wk=wk,s=3)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(ks, name='k'))

k,10,20,30,40,50
Weeks,,,,,
1,0.155932,0.191525,0.218644,0.232203,0.238983
2,0.140678,0.228814,0.257627,0.266102,0.283051
4,0.157627,0.194915,0.269492,0.315254,0.330508
8,0.152542,0.174576,0.267797,0.294915,0.332203
12,0.133898,0.184746,0.240678,0.281356,0.320339
24,0.096610,0.128814,0.169492,0.188136,0.225424
52,0.098305,0.105085,0.128814,0.144068,0.174576
104,0.083051,0.098305,0.108475,0.128814,0.140678


In [ ]:
#ks and scores
ks = range(10,51,10)
rates = range(1,6)
scores = []
for k in ks:
    for rate in rates:   
        ratings_by_user = user_recommender(ratings_trainings, users, k=k, wk=52,s=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 5)).T, index=pd.Index(rates, name='rates'), columns=pd.Index(ks, name='k'))

k,10,20,30,40,50
rates,,,,,
1,0.098305,0.105085,0.128814,0.132203,0.169492
2,0.098305,0.105085,0.128814,0.132203,0.169492
3,0.098305,0.105085,0.128814,0.144068,0.174576
4,0.098305,0.105085,0.142373,0.174576,0.189831
5,0.000000,0.001695,0.001695,0.005085,0.008475


In [ ]:
ratings_by_user = user_recommender(ratings_trainings, users, k=50, wk=8,s=3)
score = evaluate(ratings_testings_by_user, ratings_by_user)
print(score)

0.33220338983050846


### Item-based

In [ ]:
#score and wks
rates = range(1,6)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for rate in rates:
    for wk in wks:    
        ratings_by_user = item_recommender(ratings_trainings, users, k=10, wk=wk, s=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("s = %d, weeks = %d, score = %.5f" %(rate, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(rates, name='rate'))

rate,1,2,3,4,5
Weeks,,,,,
1,0.155932,0.155932,0.155932,0.159322,0.035593
2,0.140678,0.140678,0.140678,0.072881,0.016949
4,0.157627,0.157627,0.157627,0.035593,0.022034
8,0.152542,0.152542,0.152542,0.030508,0.028814
12,0.133898,0.133898,0.133898,0.020339,0.011864
24,0.096610,0.096610,0.096610,0.016949,0.011864
52,0.096610,0.096610,0.096610,0.096610,0.000000
104,0.081356,0.081356,0.081356,0.094915,0.000000


In [ ]:
#wks and ks
ks = range(10,51,10)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for k in ks:
    for wk in wks:    
        ratings_by_user = item_recommender(ratings_trainings, users, k=k, wk=wk,s=3)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(ks, name='k'))

k,10,20,30,40,50
Weeks,,,,,
1,0.155932,0.191525,0.218644,0.232203,0.238983
2,0.140678,0.228814,0.257627,0.266102,0.283051
4,0.157627,0.194915,0.269492,0.315254,0.330508
8,0.152542,0.174576,0.267797,0.294915,0.332203
12,0.133898,0.183051,0.238983,0.279661,0.318644
24,0.096610,0.130508,0.169492,0.188136,0.225424
52,0.096610,0.101695,0.127119,0.142373,0.172881
104,0.081356,0.096610,0.105085,0.125424,0.137288


In [ ]:
#ks and scores
ks = range(10,51,10)
rates = range(1,6)
scores = []
for k in ks:
    for rate in rates:   
        ratings_by_user = item_recommender(ratings_trainings, users, k=k, wk=52,s=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 5)).T, index=pd.Index(rates, name='rates'), columns=pd.Index(ks, name='k'))

k,10,20,30,40,50
rates,,,,,
1,0.09661,0.101695,0.127119,0.130508,0.167797
2,0.09661,0.101695,0.127119,0.130508,0.167797
3,0.09661,0.101695,0.127119,0.142373,0.172881
4,0.09661,0.103390,0.142373,0.172881,0.188136
5,0.00000,0.001695,0.003390,0.006780,0.010169


In [ ]:
ratings_by_user = item_recommender(ratings_trainings, users, k=50, wk=8,s=3)
score = evaluate(ratings_testings_by_user, ratings_by_user)
print(score)

0.33220338983050846


### 套件surprise

In [18]:
#score and wks
rates = range(1,6)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for rate in rates:
    for wk in wks:    
        ratings_by_user = surprise_recommender(ratings_trainings, users, k=10, wk=wk, s=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("s = %d, weeks = %d, score = %.5f" %(rate, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(rates, name='rate'))

rate,1,2,3,4,5
Weeks,,,,,
1,0.010169,0.010169,0.010169,0.010169,0.010169
2,0.003390,0.003390,0.003390,0.003390,0.003390
4,0.001695,0.001695,0.001695,0.001695,0.001695
8,0.001695,0.001695,0.001695,0.001695,0.001695
12,0.003390,0.003390,0.003390,0.003390,0.003390
24,0.001695,0.001695,0.001695,0.001695,0.001695
52,0.001695,0.001695,0.001695,0.001695,0.001695
104,0.001695,0.001695,0.001695,0.001695,0.001695


In [19]:
#wks and ks
ks = range(10,51,10)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for k in ks:
    for wk in wks:    
        ratings_by_user = surprise_recommender(ratings_trainings, users, k=k, wk=wk,s=3)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(ks, name='k'))

k,10,20,30,40,50
Weeks,,,,,
1,0.010169,0.011864,0.027119,0.054237,0.062712
2,0.003390,0.005085,0.013559,0.030508,0.059322
4,0.001695,0.006780,0.006780,0.018644,0.033898
8,0.001695,0.005085,0.010169,0.025424,0.025424
12,0.003390,0.005085,0.005085,0.018644,0.028814
24,0.001695,0.001695,0.010169,0.016949,0.020339
52,0.001695,0.001695,0.001695,0.003390,0.008475
104,0.001695,0.001695,0.001695,0.001695,0.003390


In [20]:
#ks and scores
ks = range(10,51,10)
rates = range(1,6)
scores = []
for k in ks:
    for rate in rates:   
        ratings_by_user = surprise_recommender(ratings_trainings, users, k=k, wk=52,s=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        #print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 5)).T, index=pd.Index(rates, name='rates'), columns=pd.Index(ks, name='k'))

k,10,20,30,40,50
rates,,,,,
1,0.011864,0.013559,0.018644,0.018644,0.022034
2,0.011864,0.013559,0.018644,0.018644,0.022034
3,0.011864,0.013559,0.018644,0.018644,0.022034
4,0.011864,0.013559,0.018644,0.018644,0.022034
5,0.011864,0.013559,0.018644,0.018644,0.022034
